In [1]:
# work with saved historic data to minimize the number of api requests

import pandas as pd
import os

symbol = 'btc'

#tick_data = pd.read_csv('online_trading_data.csv', index_col=1, infer_datetime_format=True, parse_dates=True)

tick_data = pd.read_csv(symbol+'_USD_2021-05-14.csv', index_col=1, infer_datetime_format=True, parse_dates=True)

#tick_data.iloc[:, 0] = pd.to_datetime(tick_data.iloc[:, 0], infer_datetime_format=True, unit='s')

#tick_data.set_index('time', inplace=True)

tick_data.sort_index(inplace=True)

# implement re-sampling
'''
bar lengths are in seconds
5 min: 300s, 10 min: 600s, 20 min: 1200s, 40 min: 2400s, 60min: 3600s
3 hr: 10800s, 6hr: 21600s, 9hr: 32400s, 12hr: 43200s
'''
df = tick_data['<CLOSE>']
bar = '21600s'
df = df.resample(bar, label='right').last().ffill()
df.head()
pd.plotting.autocorrelation_plot(df[-60:])

<AxesSubplot:xlabel='Lag', ylabel='Autocorrelation'>

In [2]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

sub = pd.DataFrame(df)

# analyze data sub-set

i = 2 # Number of ticks to use for simple moving average (SMA)
t = str(time.time()) #datetime stamp

sub['SMA'] = sub['<CLOSE>'].rolling(i).mean().shift(i)
sub.plot(figsize=(10,6), lw=0.75, title='BTC-USD', ylabel='USD/BTC')
#plt.savefig('1-BTC-USD_SMA_{}.png'.format(t))
plt.show()
sub.head()

,<CLOSE>,SMA
Date,,
2020-12-27 06:00:00,26389.290265,NaN
2020-12-27 12:00:00,26389.290265,NaN
2020-12-27 18:00:00,26389.290265,NaN
2020-12-28 00:00:00,26389.290265,26389.290265
2020-12-28 06:00:00,26718.029463,26389.290265


In [3]:
import pandas as pd
import numpy as np

# vectorized backtesting

ptc = 0.005 # proportional transactional costs - Coinbase Pro charges 0.5% per transaction

data = pd.DataFrame()
data['close'] = sub['<CLOSE>']
data['returns'] = np.log(data['close']/data['close'].shift(1))

data.dropna(inplace=True)

data = data[-60:]

lags = 13

cols = []

for lag in range(1, lags+1):
    col = 'lag_{}'.format(lag)
    data[col] = data['returns'].shift(lag)
    cols.append(col)

data.dropna(inplace=True)

data[cols] = np.where(data[cols] >= 0, 1, 0)

data['direction'] = np.where(data['returns'] > 0, 1, -1)

data[cols + ['direction']].head()

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,direction
Date,,,,,,,,,,,,,,
2021-05-02 18:00:00,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1
2021-05-03 00:00:00,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1
2021-05-03 06:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2021-05-03 12:00:00,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1
2021-05-03 18:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1


In [39]:
data.to_csv('backtest_data.csv')

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from datetime import datetime
import pickle

model = SVC(C=1, kernel='linear', gamma='auto')

split = int(len(data) * 0.50)

train = data.iloc[:split].copy()

model.fit(train[cols], train['direction'])

# persisting the model object
t = str(time.time()) #datetime stamp
pickle.dump(model, open('algorithm-{}.pkl'.format(t), 'wb'))

In [5]:
accuracy_score(train['direction'], model.predict(train[cols]))

0.9565217391304348

In [6]:
test = data.iloc[split:].copy()

test['position'] = model.predict(test[cols])
#test['position'] = test['direction']         # naive prediction

test[['position','direction']].tail()

,position,direction
Date,,
2021-05-13 06:00:00,1,-1
2021-05-13 12:00:00,-1,-1
2021-05-13 18:00:00,-1,-1
2021-05-14 00:00:00,-1,-1
2021-05-14 06:00:00,1,1


In [7]:
accuracy_score(test['direction'], test['position'])

0.9166666666666666

In [8]:
test['strategy'] = test['position'] * test['returns']
sum(test['position'].diff() != 0)

8

In [9]:
test['strategy_tc'] = np.where(test['position'].diff() != 0, test['strategy'] - ptc, test['strategy'])

In [10]:
test[['returns', 'strategy', 'strategy_tc']].sum().apply(np.exp)

returns        0.851067
strategy       1.025448
strategy_tc    0.985240
dtype: float64

In [62]:
t = str(time.time())
test[['returns', 'strategy', 'strategy_tc']].cumsum().apply(np.exp).plot(figsize=(10,6))
plt.savefig('1-BTC-USD_StratCompare_{}.png'.format(t))
plt.show()

In [20]:
import pandas as pd
from datetime import datetime
t = str(time.time()) #datetime stamp #datetime stamp
test.to_csv('backtest_{}.csv'.format(t))